In [6]:
import random as ran
import numpy as np

def random_probabilities(num=5):
    r = [ran.random() for i in range(num)]
    s = sum(r)
    return np.array([ i/s for i in r ])

def random_matrix(num=5):
    matrix = np.zeros(shape=(5, 5))
    for i in range(num):
        matrix[i] = random_probabilities(num=num)
    return matrix

In [7]:
P = random_matrix()
Q = random_matrix()

In [24]:
from numpy.linalg import norm

def l1_distance(P, Q):
    return np.sum(np.abs(P - Q))

In [25]:
l1_distance(P, Q)

2.1753351010204311

In [65]:
points[np.where(np.array([0,1,1,2,0]) == 0)]

array([[ 0.25981469,  0.25999476],
       [ 0.02582477,  0.31229703]])

In [94]:
from numpy.linalg import norm
import random as ran
import numpy as np


def dist(x, c):
    return norm(x - c)


def mean(data):
    l = len(data[0])
    c = [0.] * l
    n = 0
    for x in data:
        for i, v in enumerate(x):
            c[i] += v
        n += 1
    for i in range(l):
        c[i] /= n
    return np.array(c)


class kMeans:
    def __init__(self, n_clusters=2, initial_centers=None, n_iter=100, error_tolerance=1e-6):
        if initial_centers is None:
            self.centers = np.vstack([[0, 0], [1,1]]).astype(np.float64)
        else:
            self.centers = initial_centers

        self.n_clusters = n_clusters
        self.n_iter = n_iter
        self.error_tolerance = error_tolerance

    def run(self, data):
        cluster = [None] * len(data)
        for _ in range(self.n_iter):
            for i, x in enumerate(data):
                distances = np.array([dist(x, kmeans.centers[i]) for i in range(self.n_clusters)])
                cluster[i] = np.argmin(distances)

            cluster = np.array(cluster)
            errors = []
            for j, c in enumerate(self.centers):
                members = data[np.where(cluster == j)]
                new_center = mean(members)
                errors.append(dist(new_center, self.centers[j]))
                self.centers[j] = new_center

            if max(errors) < self.error_tolerance:
                break

    def classify(self, data_point):
        return min(range(self.n_clusters), key=lambda p: dist(data_point, self.centers[p]))

In [95]:
kmeans = kMeans()

In [96]:
points = np.vstack([[ran.random(),ran.random()]  for _ in range(100)])

In [97]:
kmeans.run(points)

In [98]:
kmeans.centers

array([[ 0.25791722,  0.46907925],
       [ 0.80531691,  0.48697692]])

https://stats.stackexchange.com/questions/280885/estimate-the-kullback-leibler-kl-divergence-with-monte-carlo

In [14]:
from math import log

def monte_carlo_distance(P, Q, n_samples_per_cluster=100):
    x, y = P.shape
    assert x == y
    
    total_distance = 0
    for cluster in range(x):
        cum_sum = P[cluster].cumsum()
        for i in range(n_samples_per_cluster):
            next_state = np.where(cum_sum >= ran.random())[0][0]
            
            total_distance += log(P[cluster][next_state] / Q[cluster][next_state])
    return total_distance / (n_samples_per_cluster * x)

def kl_distance2d(P, Q):
    x, y = P.shape
    assert x == y
    
    total_distance = 0
    for cluster in range(x):
        for next_cluster in range(y):
            total_distance += P[cluster][next_cluster] * log(P[cluster][next_cluster] / Q[cluster][next_cluster])
    
    return total_distance / (x * y)

def kl_distance(P, Q):
    total_distance = 0
    for i in range(len(P)):
        total_distance += P[i] * log(P[i]/Q[i])
            
    return total_distance

def matrix_distance(P, Q, n_samples_per_cluster=100):
     return (monte_carlo_distance(P, Q, n_samples_per_cluster=n_samples_per_cluster) + 
             monte_carlo_distance(Q, P, n_samples_per_cluster=n_samples_per_cluster)) / 2

In [276]:
monte_carlo_distance(P,Q)

0.15190685142014326

In [369]:
A = np.array([[0.3, 0.4, 0.3], [0.2, 0.4, 0.4], [0.8, 0.15, 0.05]])
B = np.array([[0.3, 0.4, 0.3], [0.2, 0.5, 0.3], [0.8, 0.15, 0.05]])

print(monte_carlo_distance(A, B, n_samples_per_cluster=10000))
print(monte_carlo_distance(B, A, n_samples_per_cluster=10000))
print(kl_distance(A, B))
print(kl_distance(B, A))

0.008350960866275046
0.00823059659222286
0.00286837871723
0.00280746154684


In [278]:
matrix_distance(A, B)

0.012076042069604893

In [31]:
import numpy as np

p1 = np.array([0.5, 0.5])
p2 = np.array([0.5, 0.5])

A1 = np.array([[0.9, 0.1], [0.2, 0.8]])
A2 = np.array([[0.9, 0.1], [0.2, 0.8]])

B1 = np.array([[0.3, 0.5, 0.2], [0.6, 0.2, 0.2]])
B2 = np.array([[0.3, 0.5, 0.2], [0.6, 0.2, 0.2]])

In [32]:
t = p1.dot(A1)
for i in range(100):
    t = t.dot(A1)
t

array([ 0.66666667,  0.33333333])

In [33]:
total = 0
for i in range(len(t)):
    total += t[i] * (kl_distance(A1[i], A2[i]) + kl_distance(B1[i], B2[i]))
    
total

0.0